In [1]:
class Dyna_Q_Agent(object):
    
    def __init__(self,action_space,environment,):
        
        
        
        self.epsilon = 0.05
        self.sample = 3
        self.alpha_r = 0.3
        self.alpha_p = 0.3
        self.alpha = 0.3
        self.alpha_decay=.999
        self.eps_decay = 0.999
        self.gamma = 0.99
        
        self.Q = dict()
        self.P = dict()
        self.R = dict()
        
        self.env = environment
        self.action_space = action_space
        self.last_obs = None
        self.last_action = None
        
        
              
                
    def dyna_q_learn_and_act(self,observation,reward,done):
        
        observation = self.env.state2str(observation)
           
        """Update Q"""
        if(observation not in self.Q):
            self.Q[observation] = np.zeros(self.action_space.n)
            
        if(not(self.last_obs is None)):
            
            if(self.last_obs not in list(self.Q.keys())):
                self.Q[self.last_obs] = np.zeros(self.action_space.n)
        
            if(done):
                self.Q[self.last_obs][self.last_action] += \
                self.alpha * (reward - self.Q[self.last_obs][self.last_action])
            else:
                try:
                    m = self.Q[observation][action]
                except:
                    m=0
                self.Q[self.last_obs][self.last_action]+=\
                self.alpha*(reward+self.gamma*m - self.Q[self.last_obs][self.last_action])
                
            """Update MDP"""
        
            try:#ee
                self.R[(self.last_obs,str(self.last_action),observation)] += \
                self.alpha_r*(reward - self.R[(self.last_obs,str(self.last_action),observation)])
            except:
                self.R[(self.last_obs,str(self.last_action),observation)] = 0
                
            try:#eee
                self.P[(observation,self.last_obs,str(self.last_action))] += self.alpha_p*(1-self.P[(observation,self.last_obs,str(self.last_action))])
            except:
                self.P[(observation,self.last_obs,str(self.last_action))] = 0  
                
                
            for obs in self.Q.keys():
                if(obs != observation):
                    try: 
                        self.P[(obs,self.last_obs,str(self.last_action))] += self.alpha_p*(1-self.P[(obs,self.last_obs,str(self.last_action))])
                    except:
                        pass
        """ Sampling """
        
        for _ in range(self.sample):
            obs = np.random.choice(list(self.Q.keys()))
            act = np.random.choice(list(range(self.action_space.n)))
            sum = 0
            for s_prime in self.Q.keys():
                m = np.max(self.Q[s_prime])
                try:
                    sum += self.P[(s_prime,obs,act)]*(self.R[(obs,act,s_prime)] + self.gamma*m)
                except:
                    pass
            self.Q[obs][act] += self.alpha * (sum - self.Q[obs][act] )
            
        """"Act"""        
        if(observation in list(self.Q.keys())):
            if(np.random.random()<self.epsilon):
                maxi = np.max(self.Q[observation])
                action = np.random.choice(np.where(self.Q[observation] == maxi)[0],1)
            else:
                action = np.random.randint(self.action_space.n)
        else:
            action = np.random.randint(self.action_space.n)


        self.epsilon *= self.eps_decay
        self.alpha *= self.alpha_decay
        self.last_obs = observation
        self.last_action = action
        
        return action
